In [2]:
import requests
import pandas as pd
import numpy as np
from functools import reduce
pd.set_option('display.max_columns', None)

In [15]:
def get_nppes_data(**kwargs: str) -> list:
    '''
    Searches the NPPES API based on various fields and returns response in json format
    
    :param number: a healthcare provider's National Provider Identifier
    :param enumeration_type: the type of healthcare provider, 1: people, 2: places
    :param taxonomy_description: exact description or exact specialty
    :param first_name: a healthcare provider's first name
    :param last_name: a healthcare provider's last name
    :param organization_name: a healthcare organization's name
    :param address_purpose: the type of address (location, mailing, primary, or specialty)
    :param city: the city a healthcare provider is located in
    :param state: the state a healthcare provider is located in
    :param postal_code: the zip code a healthcare provider is located in
    :param limit: limit results, default is 10 and max is 200
    '''
    search_params = locals()['kwargs']
    
    user_defined_search_params = list(search_params.keys())
    allowed_search_params = ['number', 'enumeration_type', 'taxonomy_description', 'first_name', 'last_name', 'organization_name', 'address_purpose', 'city', 'state', 'postal_code', 'limit']
    disallowed_search_params = np.setdiff1d(user_defined_search_params, allowed_search_params)
    
    if len(disallowed_search_params) > 0:
        print('Ensure your search parameters are valid. Invalid search params: ', disallowed_search_params)

    else:
        print('Searching the NPPES API...🔦')
        nppes_api_url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
        json_data = requests.get(nppes_api_url, params=search_params).json()
        return json_data

def json_data_to_df(json_data: list) -> pd.DataFrame:
    '''
    Converts json data from the NPPES API into a DataFrame
    '''
    main_results_df = pd.json_normalize(json_data['results'])[['number', 'basic.name','basic.name_prefix', 'basic.first_name', 'basic.last_name', 'basic.middle_name', 'basic.credential', 'basic.gender']]
    addresses_df = pd.json_normalize(json_data['results'], 'addresses', 'number')
    taxonomies_df = pd.json_normalize(json_data['results'], 'taxonomies', 'number')

    dataframes_to_merge = [main_results_df, addresses_df, taxonomies_df]

    print('Complete.')
    return reduce(lambda left,right: pd.merge(left,right,on=['number'],
                                        how='outer'), dataframes_to_merge)

        
def nppes_df(**kwargs: str) -> pd.DataFrame:
    '''
    Searches the NPPES API based on various fields and returns a DataFrame with the results
    
    :param number: a healthcare provider's National Provider Identifier
    :param enumeration_type: the type of healthcare provider, 1: people, 2: places
    :param taxonomy_description: exact description or exact specialty
    :param first_name: a healthcare provider's first name
    :param last_name: a healthcare provider's last name
    :param organization_name: a healthcare organization's name
    :param address_purpose: the type of address (location, mailing, primary, or specialty)
    :param city: the city a healthcare provider is located in
    :param state: the state a healthcare provider is located in
    :param postal_code: the zip code a healthcare provider is located in
    :param limit: limit results, default is 10 and max is 200
    '''
    json_data = get_nppes_data(**kwargs)
    
    return json_data_to_df(json_data)

In [17]:
df = nppes_df(taxonomy_description='Pharmacist', city='St. Louis')

Searching the NPPES API...🔦
{'result_count': 10, 'results': [{'enumeration_type': 'NPI-1', 'number': 1043700420, 'last_updated_epoch': 1525996800, 'created_epoch': 1525996800, 'basic': {'first_name': 'MARIA', 'last_name': 'MAULL', 'credential': 'RPH', 'sole_proprietor': 'YES', 'gender': 'F', 'enumeration_date': '2018-05-11', 'last_updated': '2018-05-11', 'status': 'A', 'name': 'MAULL MARIA'}, 'other_names': [], 'addresses': [{'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'LOCATION', 'address_type': 'DOM', 'address_1': '1039 S DUCHESNE DR', 'address_2': '', 'city': 'SAINT CHARLES', 'state': 'MO', 'postal_code': '633014836', 'telephone_number': '636-724-4848'}, {'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'MAILING', 'address_type': 'DOM', 'address_1': '668 TIMBERIDGE DR', 'address_2': '', 'city': 'SAINT CHARLES', 'state': 'MO', 'postal_code': '633033101', 'telephone_number': '314-406-0280'}], 'taxonomies': [{'code': '183500000X', 

In [18]:
df

,number,basic.name,basic.name_prefix,basic.first_name,basic.last_name,basic.middle_name,basic.credential,basic.gender,country_code,country_name,address_purpose,address_type,address_1,address_2,city,state_x,postal_code,telephone_number,fax_number,code,desc,primary,state_y,license
0,1043700420,MAULL MARIA,NaN,MARIA,MAULL,NaN,RPH,F,US,United States,LOCATION,DOM,1039 S DUCHESNE DR,,SAINT CHARLES,MO,633014836,636-724-4848,NaN,183500000X,Pharmacist,False,IL,051039423
1,1043700420,MAULL MARIA,NaN,MARIA,MAULL,NaN,RPH,F,US,United States,LOCATION,DOM,1039 S DUCHESNE DR,,SAINT CHARLES,MO,633014836,636-724-4848,NaN,183500000X,Pharmacist,True,MO,2006017831
2,1043700420,MAULL MARIA,NaN,MARIA,MAULL,NaN,RPH,F,US,United States,MAILING,DOM,668 TIMBERIDGE DR,,SAINT CHARLES,MO,633033101,314-406-0280,NaN,183500000X,Pharmacist,False,IL,051039423
3,1043700420,MAULL MARIA,NaN,MARIA,MAULL,NaN,RPH,F,US,United States,MAILING,DOM,668 TIMBERIDGE DR,,SAINT CHARLES,MO,633033101,314-406-0280,NaN,183500000X,Pharmacist,True,MO,2006017831
4,1619963758,MARTINEZ GLENN,DR.,GLENN,MARTINEZ,ERIC,"BS PHARM, PHARM.D.",M,US,United States,LOCATION,DOM,JOHN COCHRAN VA MEDICAL CENTER,915 NORTH GRAND,ST. LOUIS,MO,63106,314-289-6339,NaN,183500000X,Pharmacist,True,FL,PS 39822
5,1619963758,MARTINEZ GLENN,DR.,GLENN,MARTINEZ,ERIC,"BS PHARM, PHARM.D.",M,US,United States,MAILING,DOM,375 BECKLEY PL,,SAINT CHARLES,MO,633041030,636-577-7369,NaN,183500000X,Pharmacist,True,FL,PS 39822
6,1871583799,THEKKEPAT NIMITA,DR.,NIMITA,THEKKEPAT,NaN,"PHARM D., MBA",F,US,United States,LOCATION,DOM,9568 CHANCELORSVILLE DRIVE,,ST. LOUIS,MO,631263311,540-664-5371,NaN,1835P1200X,Pharmacist Pharmacotherapy,False,VA,0202206608
7,1871583799,THEKKEPAT NIMITA,DR.,NIMITA,THEKKEPAT,NaN,"PHARM D., MBA",F,US,United States,LOCATION,DOM,9568 CHANCELORSVILLE DRIVE,,ST. LOUIS,MO,631263311,540-664-5371,NaN,1835P1200X,Pharmacist Pharmacotherapy,True,MO,2006031391
8,1871583799,THEKKEPAT NIMITA,DR.,NIMITA,THEKKEPAT,NaN,"PHARM D., MBA",F,US,United States,MAILING,DOM,9568 CHANCELORSVILLE DR,,SAINT LOUIS,MO,631263311,540-664-5371,NaN,1835P1200X,Pharmacist Pharmacotherapy,False,VA,0202206608
9,1871583799,THEKKEPAT NIMITA,DR.,NIMITA,THEKKEPAT,NaN,"PHARM D., MBA",F,US,United States,MAILING,DOM,9568 CHANCELORSVILLE DR,,SAINT LOUIS,MO,631263311,540-664-5371,NaN,1835P1200X,Pharmacist Pharmacotherapy,True,MO,2006031391


In [152]:
# for d in json_data['results']:
#     try:
#         d['practiceLocations']
#     except KeyError:
#         d.update({'practiceLocations': []}) 

main_results_df = pd.json_normalize(json_data['results'])[['number', 'basic.name','basic.name_prefix', 'basic.first_name', 'basic.last_name', 'basic.middle_name', 'basic.credential', 'basic.gender']]
addresses_df = pd.json_normalize(json_data['results'], 'addresses', 'number')
# practice_locations_df = pd.json_normalize(json_data['results'], 'practiceLocations', 'number')
taxonomies_df = pd.json_normalize(json_data['results'], 'taxonomies', 'number')

dataframes_to_merge = [main_results_df, addresses_df, practice_locations_df, taxonomies_df]

nppes_results = reduce(lambda left,right: pd.merge(left,right,on=['number'],
                                            how='outer'), dataframes_to_merge)

In [153]:
nppes_results

,number,basic.name,basic.name_prefix,basic.first_name,basic.last_name,basic.middle_name,basic.credential,basic.gender,country_code_x,country_name_x,address_purpose,address_type_x,address_1_x,address_2_x,city_x,state_x,postal_code_x,telephone_number_x,fax_number_x,address_type_y,address_1_y,address_2_y,city_y,state_y,postal_code_y,country_code_y,country_name_y,fax_number_y,telephone_number_y,update_date,code,desc,primary,state,license
0,1881949022,MOORE JAIME,MRS.,JAIME,MOORE,LEIGH,LMHC,F,US,United States,LOCATION,DOM,193 LOCUST ST STE 2,,NORTHAMPTON,MA,01060,413-584-8700,413-584-1714,DOM,170 UNIVERSITY DR STE 101,,AMHERST,MA,010022272,US,United States,413-584-1714,413-584-8700,2018-08-27 09:08,101YM0800X,Counselor Mental Health,True,,
1,1881949022,MOORE JAIME,MRS.,JAIME,MOORE,LEIGH,LMHC,F,US,United States,MAILING,DOM,764 PENDLETON AVE,,CHICOPEE,MA,010202950,413-530-6099,NaN,DOM,170 UNIVERSITY DR STE 101,,AMHERST,MA,010022272,US,United States,413-584-1714,413-584-8700,2018-08-27 09:08,101YM0800X,Counselor Mental Health,True,,
2,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,LOCATION,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,False,GA,054866
3,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,LOCATION,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,False,TX,K8666
4,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,LOCATION,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,True,CT,053916
5,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,MAILING,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,False,GA,054866
6,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,MAILING,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,False,TX,K8666
7,1437100542,MOORE JAMES,NaN,JAMES,MOORE,E,MD PHD,M,US,United States,MAILING,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2080N0001X,Pediatrics Neonatal-Perinatal Medicine,True,CT,053916
8,1639304041,MOORE JAMES,NaN,JAMES,MOORE,P,MD,M,US,United States,LOCATION,DOM,4500 LOWER RIVER RD,,LEWISTON,NY,140921057,716-754-4998,716-754-4998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207R00000X,Internal Medicine,True,NY,091628
9,1639304041,MOORE JAMES,NaN,JAMES,MOORE,P,MD,M,US,United States,MAILING,DOM,4500 LOWER RIVER RD,,LEWISTON,NY,140921057,716-754-4998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207R00000X,Internal Medicine,True,NY,091628


In [126]:
json_data['results']

[{'enumeration_type': 'NPI-1',
  'number': 1881949022,
  'last_updated_epoch': 1571827031,
  'created_epoch': 1342137600,
  'basic': {'name_prefix': 'MRS.',
   'first_name': 'JAIME',
   'last_name': 'MOORE',
   'middle_name': 'LEIGH',
   'credential': 'LMHC',
   'sole_proprietor': 'NO',
   'gender': 'F',
   'enumeration_date': '2012-07-13',
   'last_updated': '2019-10-23',
   'status': 'A',
   'name': 'MOORE JAIME'},
  'other_names': [{'code': '1',
    'type': 'Former Name',
    'last_name': 'LAPIERRE',
    'first_name': 'JAIME',
    'middle_name': 'LEIGH',
    'prefix': 'MISS',
    'credential': 'LMHC'}],
  'addresses': [{'country_code': 'US',
    'country_name': 'United States',
    'address_purpose': 'LOCATION',
    'address_type': 'DOM',
    'address_1': '193 LOCUST ST STE 2',
    'address_2': '',
    'city': 'NORTHAMPTON',
    'state': 'MA',
    'postal_code': '01060',
    'telephone_number': '413-584-8700',
    'fax_number': '413-584-1714'},
   {'country_code': 'US',
    'country

In [41]:
df

,country_code,country_name,address_purpose,address_type,address_1,address_2,city,state,postal_code,telephone_number,fax_number,number
0,US,United States,LOCATION,DOM,193 LOCUST ST STE 2,,NORTHAMPTON,MA,01060,413-584-8700,413-584-1714,1881949022
1,US,United States,MAILING,DOM,764 PENDLETON AVE,,CHICOPEE,MA,010202950,413-530-6099,NaN,1881949022
2,US,United States,LOCATION,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,1437100542
3,US,United States,MAILING,DOM,282 WASHINGTON ST,,HARTFORD,CT,061063322,860-545-9778,860-545-8959,1437100542
4,US,United States,LOCATION,DOM,4500 LOWER RIVER RD,,LEWISTON,NY,140921057,716-754-4998,716-754-4998,1639304041
5,US,United States,MAILING,DOM,4500 LOWER RIVER RD,,LEWISTON,NY,140921057,716-754-4998,NaN,1639304041
6,US,United States,LOCATION,DOM,2808 NORTH AVE FL 3,,GRAND JUNCTION,CO,815015155,970-241-6023,970-242-8330,1174960074
7,US,United States,MAILING,DOM,715 HORIZON DR STE 225,,GRAND JUNCTION,CO,815068743,970-683-7131,NaN,1174960074
8,US,United States,LOCATION,DOM,599 TOMALES RD,,PETALUMA,CA,949525002,260-445-3880,NaN,1467698662
9,US,United States,MAILING,DOM,599 TOMALES RD,,PETALUMA,CA,949525002,NaN,NaN,1467698662
